In [1]:
from ipywidgets import interact
from brian2 import * #type: ignore
import NeuronEquations
import BrianHF
from ipywidgets import interact

WARNING    /home/bernard/Desktop/IIT_Thesis/ERO-SNN/BrianHF.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
 [py.warnings]


In [2]:
@interact(tau=(0.1,10.0), tauSpi = (0.1,10.0), beta=(0.1, 5.0, 0.1), Wi=(0.0, 50.0, 0.1), Wk=(-10.0, 0.0, 0.1), vt=(0.1, 1.0, 0.1), vr=(0.0, 1.1, 0.1))
def simulate_neurons(tau=0.1, tauSpi = 0.5, vt=1.0, vr=0.0, beta=1.5, Wi=11.3, Wk=-2.0):
    start_scope()
    
    defaultclock.dt = 0.01*ms

    grid_size_x = 5 # Number of neurons in the x direction
    grid_size_y = 5
    N_Neurons = grid_size_x * grid_size_y # Number of neurons\
        
        
    # Neuron Equation
    Eqs_Neurons = NeuronEquations.EQ_SCM_IF

    # Neuron Parameters
    Neuron_Params = {'tau': tau*ms, 'tauSpi': tauSpi*ms, 'vt': vt, 'vr': vr, 'P': 0, 'incoming_spikes': 0}
    # Synapse Parameters
    Syn_Params = {'Num_Neighbours' : 2, 'beta': beta, 'Wi': Wi, 'Wk': Wk}


    # Input generation
    indices = array([12, 10, 14, 12, 14])
    times = array([0, 0.5, 0.5, 1.0, 1.5])*ms
    G_Spikes = SpikeGeneratorGroup(N_Neurons, indices, times)
    
    # indices = np.concatenate((arange(N_Neurons), arange(5), arange(7,11)))
    # times = np.concatenate(([1]*N_Neurons, [2]*5, [3]*4))*ms
    # G_Spikes = SpikeGeneratorGroup(N_Neurons, indices, times)

    G_Neurons = NeuronGroup(N_Neurons, Eqs_Neurons, threshold='v>vt',
                            reset='''
                            v = vr
                            incoming_spikes_post = 0
                            ''',
                            refractory='0*ms',
                            events={'P_ON': 'v > vt', 'P_OFF': '(timestep(t - lastspike, dt) > timestep(dt, dt) and v <= vt)'},
                            method='euler',
                            namespace=Neuron_Params)
    G_Neurons.run_on_event('P_ON', 'P = 1' , when = 'after_thresholds')
    G_Neurons.set_event_schedule('P_OFF', when = 'before_groups')
    G_Neurons.run_on_event('P_OFF', 'P = 0', when = 'before_groups')


    

    # Generate x and y values for each neuron
    x_values = np.repeat(np.arange(grid_size_x), grid_size_y)
    y_values = np.tile(np.arange(grid_size_y), grid_size_x)

    # Assign x, y, and z values to each neuron
    G_Neurons.X = x_values
    G_Neurons.Y = y_values

    # Creating the synapses
    Syn_Input_Neurons = Synapses(G_Spikes, G_Neurons,
                                 'beta : 1 (constant)',
                                 on_pre='''
                                 ExtIn_post = beta
                                 ''',
                                 namespace=Syn_Params)
 
    # Syn_Neurons_Exc = Synapses(G_Neurons, G_Neurons,
    #                            '''
    #                            Wi : 1
    #                            ''',
    #                            on_pre='''
    #                            Exc_post = Wi
    #                            ''')

    Syn_Neurons_Inh = Synapses(G_Neurons, G_Neurons,
                               '''
                               Wi : 1
                               Wk : 1
                               ''',
                               on_pre={
                                   'pre':'incoming_spikes_post += 1; Exc_pre = Wi',
                                   'after_pre': 'Inh_post = clip(Inh_post + Wk * incoming_spikes_post/N_outgoing, Wk, 0)'},
                               method='euler', 
                               namespace=Syn_Params)

    
    # SYNAPSE CONNECTIONS:
    # Connect the first synapses from input to G_neurons on a 1 to 1 basis
    Syn_Input_Neurons.connect(condition= 'i == j')
    Syn_Input_Neurons.beta = beta
    
    # # Connect the second set of synapses from a neuron to itself (recurrent)
    # Syn_Neurons_Exc.connect(condition='i == j')
    # Syn_Neurons_Exc.Wi = Wi

    # Connect the last group of synapses from a neuron to its neighbors
    Syn_Neurons_Inh.connect(condition='i != j and (abs(X_pre - X_post) <= Num_Neighbours and abs(Y_pre - Y_post) <= Num_Neighbours)')
    Syn_Neurons_Inh.Wi = Wi
    Syn_Neurons_Inh.Wk = Wk




    # SETTING UP MONITORS:
    # Monitor the spikes
    SpikeMon_Input = SpikeMonitor(G_Spikes)
    SpikeMon_Neurons = SpikeMonitor(G_Neurons)
    P_ON_Mon = EventMonitor(G_Neurons, 'P_ON')
    P_OFF_Mon = EventMonitor(G_Neurons, 'P_OFF')

    # Monitor the state variables
    StateMon_Neurons = StateMonitor(G_Neurons, ['v', 'In', 'ExtIn', 'SystIn', 'P', 'Exc', 'Inh'], record=True)

    # RUNNING THE SIMULATION:
    # display(scheduling_summary())
    run(5*ms)

    # Plot the results
    figure()
    subplot(311)
    plot(SpikeMon_Input.t/ms, SpikeMon_Input.i, '.k')
    ylabel('Neuron index')

    # Plot the results
    subplot(312)
    plot(SpikeMon_Neurons.t/ms, SpikeMon_Neurons.i, '.k')
    ylabel('Neuron index')
    show()
    
    subplot(313)
    plot(P_OFF_Mon.t/ms, P_OFF_Mon.i, '.k')
    plot(P_ON_Mon.t/ms, P_ON_Mon.i, 'xr')
    xlabel('Time (ms)')
    ylabel('Neuron index')
    show()

    BrianHF.visualise_neurons_states(StateMon_Neurons, [3, 4, 5], 'all', overlap = True)
    BrianHF.visualise_states(StateMon_Neurons, [3, 4, 5], 'all', overlap = True)

interactive(children=(FloatSlider(value=0.1, description='tau', max=10.0, min=0.1), FloatSlider(value=0.5, des…